<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_12_Traders_Move_part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 引入依赖包

In [1]:

try: 
  import mesa
except: 
  !pip install mesa --quiet
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib inline

# 辅助函数

In [2]:
def get_distance(pos_1, pos_2):
  '''
  计算两点欧式距离
  
  在交易主体移动函数：trade.move()中被调用
  '''

  x1, y1 = pos_1
  x2, y2 = pos_2
  dx = x1 - x2
  dy = y1 - y2
  return math.sqrt(dx**2 + dy**2)

# 资源类 

In [3]:
class Sugar(mesa.Agent):
    '''
    Sugar:
    - 包含一定数量的糖
    - 每轮增长一单位数量的糖
    '''
    def __init__(self, unique_id, model, pos, max_sugar): 
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar


    def step(self):
        '''
        糖每轮增长一单位数量
        '''
        self.amount = min([self.max_sugar, self.amount+1])

In [4]:
class Spice(mesa.Agent):
  '''
  spice:
  - 包含一定数量的香料
  - 每轮增长一单位数量的香料
  '''

  def __init__(self, unique_id, model, pos, max_spice):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_spice
    self.max_spice = max_spice
  
  def step(self): 
    '''
    香料每轮增长一单位数量
    '''
    self.amount = min([self.max_spice, self.amount+1])

# 交易主体类

In [5]:
class Trader(mesa.Agent): 
    '''
    Trader:
    - 能够代谢糖和香料
    - 收集和交易糖和香料来生存与繁衍
    '''


    def __init__(self, unique_id, model, pos, moore=False, sugar=0, 
                spice=0, metabolism_sugar=0, metabolism_spice=0, 
                vision=0):
      super().__init__(unique_id, model)
      self.pos = pos
      self.moore = moore
      self.sugar = sugar
      self.spice = spice
      self.metabolism_sugar = metabolism_sugar
      self.metabolism_spice = metabolism_spice
      self.vision = vision

    
    def get_sugar(self, pos):
      '''
      在self.get_sugar_amount()中被调用
      '''

      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
        if type(agent) is Sugar:
          return agent
      return None
    
    
    def get_sugar_amount(self,pos):
      '''
      在self.move()中self.calculate_welfare()部分被调用
      '''

      sugar_patch = self.get_sugar(pos)
      if sugar_patch:
        return sugar_patch.amount
      return 0
    
    def get_spice(self, pos):
      '''
      在 self.get_spice_amount()中被调用
      '''

      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell: 
        if type(agent) is Spice: 
          return agent
      return None

    def get_spice_amount(self, pos):
      '''
      在self.move()中self.calculate_welfare()部分被调用
      '''

      spice_patch = self.get_spice(pos)
      if spice_patch: 
        return spice_patch.amount
      return 0
    
      
    def is_occupied_by_other(self,pos):
      '''
      辅助函数1：在self.move()的第一部分被调用
      '''

      if pos == self.pos: 
        # 主体本来的位置被视作未被占据，因为主体可以选择待在原地不动
        return False
      # 获取pos位置的所有agent
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell: 
        # 判断是否被其它交易主体占据
        if isinstance(a, Trader): 
          return True
      return False
    
    def calculate_welfare(self, sugar, spice):
      '''
      辅助函数2: 在self.move()的第二部分被调用
      '''

      # calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      # Cobb-Douglas functional form    
      return sugar**(self.metabolism_sugar/m_total) * spice**(
          self.metabolism_spice/m_total)
    

    def is_starved(self):
        '''
        辅助函数3: 在elf.maybe_die()中被调用
        '''

        return (self.sugar <= 0) or (self.spice <= 0)

    
    
    ######################################################################
    #                                                                    #  
    #                      MAIN TRADE FUNCTIONS                          #  
    #                                                                    #
    ######################################################################
    
    
    def move(self):
      '''
      通过四个步骤，为交易主体代理确定最优移动的功能
      Function for trader agent to identify optimal move for each step in 4 parts
      1 - 找出所有可能的走法
      2 - 确定哪一步能使福利最大化
      3 - 找到最近的最佳选择
      4 - 移动
      '''

      # 1. 找出所有可能的走法

      neighbors = [i
                  for i in self.model.grid.get_neighborhood(
                    self.pos, self.moore, True, self.vision  
                  ) if not self.is_occupied_by_other(i)]

      # 2. 确定哪一步能使福利最大化

      welfares = [
          self.calculate_welfare(
              self.sugar + self.get_sugar_amount(pos),
              self.spice + self.get_spice_amount(pos)) 
          for pos in neighbors
      ]

      # 3. 找到最近的最佳选择

      # 找到其中最大的福利
      max_welfare = max(welfares)
      # 找出最大福利的在所有可能的走法中的索引（最大值可能不只一个）作为候选的索引
      candidate_indices = [i for i in range(len(welfares))
                          if math.isclose(welfares[i], max_welfare)]

      # 将索引转换为坐标作为候选
      candidates = [neighbors[i] for i in candidate_indices]

      # 找到最近的最佳选择（也可能不止一个）
      min_dist = min(get_distance(self.pos, pos) for pos in candidates)

      final_candidates = [ pos for pos in candidates
                        if math.isclose(get_distance(self.pos, pos), min_dist, rel_tol=1e-02
                                        )]
      self.random.shuffle(final_candidates) # 随机打乱顺序，以防止主体总是选择同一个方向

      # 4. 移动
      self.model.grid.move_agent(self, final_candidates[0])

    def eat(self):
      # 收获糖
      sugar_patch = self.get_sugar(self.pos)

      if sugar_patch:
          self.sugar += sugar_patch.amount
          sugar_patch.amount = 0
      # 代谢糖
      self.sugar -= self.metabolism_sugar

      # 收获香料
      spice_patch = self.get_spice(self.pos)

      if spice_patch:
          self.spice += spice_patch.amount
          spice_patch.amount = 0
      # 代谢香料
      self.spice -= self.metabolism_spice

    def maybe_die(self):
      '''
      消耗完糖或者香料后，交易主体死亡
      '''

      if self.is_starved(): 
        print(self.unique_id, self.model.schedule.get_type_count(Trader))
        self.model.grid.remove_agent(self)
        self.model.schedule.remove(self)
        print(self.unique_id, self.model.schedule.get_type_count(Trader))



# 模型类

In [6]:
class SugarscapeG1mt(mesa.Model):
  '''
  通过Traders (GImt)来运行Sugarscape的模型
  来自 Axtell和Epstein《Growing Artifical Societies》
  '''
  
  
  def __init__(self, width=50,height=50, initial_population=200,
               endowment_min=25, endowment_max=50, metabolism_min=1,
               metabolism_max=5, vision_min=1, vision_max=5):
    
    # 初始化糖景的宽度和高度
    self.width = width
    self.height = height 
    # 初始化交易主体属性
    self.initial_population = initial_population
    self.endowment_min = endowment_min
    self.endowment_max = endowment_max
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max

    # 初始化mesa的日程表
    self.schedule = mesa.time.RandomActivationByType(self)    
    # 初始化mesa的网格类
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    # 从辅助资料中读取糖和香料的分布
    sugar_distribution =np.genfromtxt("sugar-map.txt")
    spice_distribution = np.flip(sugar_distribution, 1)
    
    agent_id = 0
    for _,x,y in self.grid.coord_iter():
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y), max_sugar)
        self.schedule.add(sugar)
        self.grid.place_agent(sugar, (x,y))
        agent_id += 1
    
      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.schedule.add(spice)
        self.grid.place_agent(spice, (x,y))
        agent_id += 1    

    for i in range(self.initial_population):
      # 得到随机的坐标
      x = self.random.randrange(self.width)
      y = self.random.randrange(self.height)
      # see Growing Artificial Societies p. 108 for initialization
      # 给予主体初始的禀赋
      sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      # 给予主体初始代谢
      metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      # 给予主体视野
      vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
      # 实例化交易主体
      trader = Trader(agent_id, 
                      self,
                      (x,y),
                      moore = False, 
                      sugar = sugar, 
                      spice = spice, 
                      metabolism_sugar = metabolism_sugar, 
                      metabolism_spice = metabolism_spice, 
                      vision = vision)
      # 放置主体到网格中
      self.grid.place_agent(trader, (x,y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self):
    '''
    独特的step函数，可分阶段激活糖和香料 然后随机激活交易主体
    '''    
    # step Sugar agents
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    # step Spice agents
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()

    # step trader agents
    # 为了考虑代理的死亡和移除，我们需要一个单独的数据结构来进行迭代
    trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
    self.random.shuffle(trader_shuffle)

    for agent in trader_shuffle: 
      agent.move()
      agent.eat()
      agent.maybe_die()
      
    self.schedule.steps += 1 # 对于数据收集器(data collector)跟踪步数很重要

  def run_model(self, step_count=1000):

    for i in range(step_count):
      self.step()

# 运行Sugarscape模型

In [7]:
model = SugarscapeG1mt()
model.run_model(step_count=15)


4177 200
4177 199
4196 199
4196 198
4144 198
4144 197
4214 197
4214 196
4225 196
4225 195
4326 195
4326 194
4197 194
4197 193
4219 193
4219 192
4172 192
4172 191
4274 191
4274 190
4150 190
4150 189
4251 189
4251 188
4152 188
4152 187
4143 187
4143 186
4180 186
4180 185
4327 185
4327 184
4141 184
4141 183
4279 183
4279 182
4200 182
4200 181
4288 181
4288 180
4259 180
4259 179
4277 179
4277 178
4244 178
4244 177
4236 177
4236 176
4289 176
4289 175
4201 175
4201 174
4188 174
4188 173
4294 173
4294 172
4304 172
4304 171
4168 171
4168 170
4323 170
4323 169
4336 169
4336 168
4291 168
4291 167
4186 167
4186 166
4332 166
4332 165
4183 165
4183 164
4160 164
4160 163
4154 163
4154 162
4212 162
4212 161
4255 161
4255 160
4176 160
4176 159
4264 159
4264 158
4281 158
4281 157
4220 157
4220 156
4328 156
4328 155
4155 155
4155 154
4234 154
4234 153
4157 153
4157 152
4295 152
4295 151
4282 151
4282 150
4296 150
4296 149
4318 149
4318 148
4263 148
4263 147
4250 147
4250 146
4330 146
4330 145
4162 145
4